In [10]:
import sys
sys.path.append('../Main-Version/')

from AD_analysis import *
import Atmospheric_diffraction as atm_shift
import numpy as np
import matplotlib as mpl
import Integ_analysis as int_analysis
import Transmission_calculation as trans_calc
import pandas as pd

In [11]:
#Following function plots integration transmission average for parallel observations
def plot_parallel_integ(analysis,guide,aperture_VIS,aperture_NIR,VIS_band,normalise="no AD",method="numerical moffat",
                        aperture="hexagons",sample_val=[1*u.nm,1*u.nm],scale_val=0.01):  
    if VIS_band=="G":
        VIS_band="V"
    weights_VIS = np.arange(0, len(aperture_VIS)+2)
    norm_VIS = mpl.colors.Normalize(vmin=min(weights_VIS), vmax=max(weights_VIS))
    cmap_VIS = mpl.cm.ScalarMappable(norm=norm_VIS, cmap=mpl.cm.Blues)

    weights_NIR = np.arange(0, len(aperture_NIR)+2)
    norm_NIR = mpl.colors.Normalize(vmin=min(weights_NIR), vmax=max(weights_NIR))
    cmap_NIR= mpl.cm.ScalarMappable(norm=norm_NIR, cmap=mpl.cm.Reds)
       
    analysis.load_band("LR","VIS",VIS_band,VIS_band,sampling=sample_val[0])
    analysis.make_aperture(aperture,method,scale_val)
    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,aperture_VIS,guide)
    
    plt.figure(figsize=[7,5])
    if normalise == "none":
        for count,i in enumerate(integ_transmissions):
            plt.plot(analysis.output['wavelengths'],np.array(i),label='Aperture Centre = {}'.format(aperture_VIS[count]),color=cmap_VIS.to_rgba(len(aperture_VIS)-count))
        plt.plot(analysis.output['wavelengths'],opt_transmission,color='black',linestyle='--')
        
    if normalise == "no AD":
        for count,i in enumerate(integ_transmissions):
            plt.plot(analysis.output['wavelengths'],np.array(i)/np.array(opt_transmission),label='Aperture Centre = {}'.format(aperture_VIS[count]),color=cmap_VIS.to_rgba(len(aperture_VIS)-count))

    analysis.load_band("LR","NIR","IY","H",sampling=sample_val[1])
    analysis.make_aperture(aperture,method,scale_val)
    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,aperture_NIR,guide)

    if normalise == "none":
        for count,i in enumerate(integ_transmissions):
            plt.plot(analysis.output['wavelengths'],np.array(i),label='Aperture Centre = {}'.format(aperture_NIR[count]),color=cmap_NIR.to_rgba(len(aperture_NIR)-count))
        plt.ylabel("Mean Transmission")        
        plt.plot(analysis.output['wavelengths'],opt_transmission,label='No AD Transmission',color='black',linestyle='--')
    if normalise == "no AD":
        for count,i in enumerate(integ_transmissions):
            plt.plot(analysis.output['wavelengths'],np.array(i)/np.array(opt_transmission),label='Aperture Centre = {}'.format(aperture_NIR[count]),color=cmap_NIR.to_rgba(len(aperture_NIR)-count))
        plt.axhline(y=1,label='No AD Transmission, {}'.format(analysis.input['method']),color='black',linestyle='--')
        plt.ylabel("Mean Transmission Relative to No AD")
        
    plt.ylim(0,1.1)
    plt.xlabel("Wavelength (nm)")
    plt.axvline(x=guide.value*1000,color='black',linewidth=0.7,label='Guide Wavelength = {}'.format(guide))
    #if analysis.input['HA_range'] != []:
        #plt.title("Guide = {}, HA {}-{}, Dec = {}, Centre at = {}h".format(guide,analysis.input['HA_range'][0],analysis.input['HA_range'][-1],analysis.input['targ_dec'],analysis.input['HA_range'][analysis.input['centred_on']]))
    plt.legend()

In [12]:
def optimise_parallel_min(analysis,guide_options,VIS_aperture_centre_options,NIR_aperture_centre_options,VIS_band,scale_val=0.01):
    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=scale_val)
    VIS_metrics=[] #will be of the form [[[G1 A1 min, max, mean],[G1 A2 min, max, mean]...][[G2 A1 min, max, mean...]...]...]
    #first index is guide, second index is aperture centre, third index in min, max, or mean
    for guide in guide_options:
        integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,VIS_aperture_centre_options,guide)
        VIS_metrics.append(int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission)))
    
    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=scale_val)
    NIR_metrics=[] #will be of the form [[[G1 A1 min, max, mean],[G1 A2 min, max, mean]...][[G2 A1 min, max, mean...]...]...]
    #first index is guide, second index is aperture centre, third index in min, max, or mean
    for guide in guide_options:
        integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,NIR_aperture_centre_options,guide)
        NIR_metrics.append(int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission)))
    
    best_min=0
    best_VIS_min=0
    best_NIR_min=0

    for i in range(0,len(guide_options)):
        for o in range(0,len(VIS_aperture_centre_options)):
            for p in range(0,len(NIR_aperture_centre_options)):
                VIS_min=VIS_metrics[i][o][0]
                NIR_min=NIR_metrics[i][p][0]
                if VIS_min > NIR_min:
                    min_val=NIR_min
                    min_type="NIR"
                if NIR_min > VIS_min:
                    min_val=VIS_min
                    min_type="VIS"
                    
                if min_type=="VIS":
                    if min_val >= best_min and NIR_min > best_NIR_min:
                        best_min=min_val
                        best_NIR_min = NIR_min
                        best_VIS_min = VIS_min
                    
                        best_guide=guide_options[i]
                        best_VIS_aperture=VIS_aperture_centre_options[o]
                        best_NIR_aperture=NIR_aperture_centre_options[p]
                        
                        best_VIS_metrics=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                        best_NIR_metrics=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
                                                
                if min_type=="NIR":
                    if min_val >= best_min and VIS_min>best_VIS_min:
                        best_min=min_val
                        best_NIR_min = NIR_min
                        best_VIS_min = VIS_min
                    
                        best_guide=guide_options[i]
                        best_VIS_aperture=VIS_aperture_centre_options[o]
                        best_NIR_aperture=NIR_aperture_centre_options[p]
                        
                        best_VIS_metrics=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                        best_NIR_metrics=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
                    
    best_metric=best_min               
    best_metric_combo=[best_metric,best_VIS_metrics,best_NIR_metrics]

    return [best_guide,best_VIS_aperture,best_NIR_aperture,best_metric_combo]

In [13]:
# decs = np.arange(-80,25,5)
# HAs = np.arange(0,5,1)
# VIS_bands=["B","G","R"]
# PSFs=['moffat','durham']

# total_decs=[]
# for p in range(0,len(PSFs)):
#     for o in range(0,len(VIS_bands)):
#         for dec in decs:
#             for i in range(0,len(HAs)):         
#                 total_decs.append(dec)
# total_HAs=[]
# for p in range(0,len(PSFs)):
#     for o in range(0,len(VIS_bands)):   
#         for i in range(0,len(decs)):
#             for HA in HAs:
#                 total_HAs.append(HA)
# total_bands=[]
# for p in range(0,len(PSFs)):
#     for band in VIS_bands:
#         for i in range(0,len(decs)):
#             for o in range(0,len(HAs)):
#               total_bands.append(band)
# total_PSFs=[]
# for PSF in PSFs:
#     for o in range(0,len(VIS_bands)):
#         for i in range(0,len(HAs)):
#             for p in range(0,len(decs)):
#                 total_PSFs.append(PSF)  

# data = {'HA mid': total_HAs,'Dec': total_decs,'VIS band': total_bands,'PSF': total_PSFs}
# metrics=pd.DataFrame(data)

# metrics['Guide']=np.nan
# metrics['VIS Centre']=np.nan
# metrics['NIR Centre']=np.nan

# metrics['VIS Min']=np.nan
# metrics['VIS Max']=np.nan
# metrics['VIS Throughput']=np.nan

# metrics['NIR Min']=np.nan
# metrics['NIR Max']=np.nan
# metrics['NIR Throughput']=np.nan

# metrics['Metric']=np.nan

# metrics.to_csv("file_name.csv",index=False)


In [ ]:
VIS_B=[.390,.458]
VIS_V=[.450,.591] 
VIS_R=[.586,.770]
NIR_All=[.770,1.857]
indexes=np.arange(0,105,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_min.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,11)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    v1=optimise_parallel_min(analysis,np.arange(.4,.6,.01)*u.micron,np.arange(VIS_B[0],VIS_B[1],.01)*u.micron,np.arange(.9,1.2,.01)*u.micron,'B',scale_val=0.1)

    best_guide,best_VIS_aperture,best_NIR_aperture,best_metric_combo=v1[0],v1[1],v1[2],v1[3]

    aperture_VIS=[round(best_VIS_aperture.value,2)]*u.micron
    aperture_NIR=[round(best_NIR_aperture.value,2)]*u.micron
    guide=round(best_guide.value,2)*u.micron

    metrics.at[i,'Guide']=guide.value
    metrics.at[i,'VIS Centre']=aperture_VIS.value
    metrics.at[i,'NIR Centre']=aperture_NIR.value
    metrics.at[i,'VIS Min']=best_metric_combo[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo[2][2]
    metrics.at[i,'Metric']=best_metric_combo[0]
    metrics.to_csv("generic_test_cases_min.csv",index=False)

In [27]:
indexes=np.arange(105,210,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_min.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,11)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    v1=optimise_parallel_min(analysis,np.arange(.3,2.01,.01)*u.micron,np.arange(VIS_V[0],.53,.01)*u.micron,np.arange(.95,1.15,.01)*u.micron,'V',scale_val=0.1)

    best_guide,best_VIS_aperture,best_NIR_aperture,best_metric_combo=v1[0],v1[1],v1[2],v1[3]

    aperture_VIS=[round(best_VIS_aperture.value,2)]*u.micron
    aperture_NIR=[round(best_NIR_aperture.value,2)]*u.micron
    guide=round(best_guide.value,2)*u.micron

    metrics.at[i,'Guide']=guide.value
    metrics.at[i,'VIS Centre']=aperture_VIS.value
    metrics.at[i,'NIR Centre']=aperture_NIR.value
    metrics.at[i,'VIS Min']=best_metric_combo[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo[2][2]
    metrics.at[i,'Metric']=best_metric_combo[0]
    metrics.to_csv("generic_test_cases_min.csv",index=False)    

204
Target goes below Horizon above/below HA of +/- 5.5h
209
Target goes below Horizon above/below HA of +/- 5.4h


In [8]:
indexes=np.arange(210,315,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_min.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,11)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    v1=optimise_parallel_min(analysis,np.arange(.5,1.6,.01)*u.micron,np.arange(.58,.72,.01)*u.micron,np.arange(.95,1.15,.01)*u.micron,'R',scale_val=0.1)

    best_guide,best_VIS_aperture,best_NIR_aperture,best_metric_combo=v1[0],v1[1],v1[2],v1[3]

    aperture_VIS=[round(best_VIS_aperture.value,2)]*u.micron
    aperture_NIR=[round(best_NIR_aperture.value,2)]*u.micron
    guide=round(best_guide.value,2)*u.micron

    metrics.at[i,'Guide']=guide.value
    metrics.at[i,'VIS Centre']=aperture_VIS.value
    metrics.at[i,'NIR Centre']=aperture_NIR.value
    metrics.at[i,'VIS Min']=best_metric_combo[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo[2][2]
    metrics.at[i,'Metric']=best_metric_combo[0]
    metrics.to_csv("generic_test_cases_min.csv",index=False)    

265
Target goes below Horizon above/below HA of +/- 6.8h
266
Target goes below Horizon above/below HA of +/- 6.8h
267
Target goes below Horizon above/below HA of +/- 6.8h
268
Target goes below Horizon above/below HA of +/- 6.8h
269
Target goes below Horizon above/below HA of +/- 6.8h
270
Target goes below Horizon above/below HA of +/- 6.6h
271
Target goes below Horizon above/below HA of +/- 6.6h
272
Target goes below Horizon above/below HA of +/- 6.6h
273
Target goes below Horizon above/below HA of +/- 6.6h
274
Target goes below Horizon above/below HA of +/- 6.6h
275
Target goes below Horizon above/below HA of +/- 6.5h
276
Target goes below Horizon above/below HA of +/- 6.5h
277
Target goes below Horizon above/below HA of +/- 6.5h
278
Target goes below Horizon above/below HA of +/- 6.5h
279
Target goes below Horizon above/below HA of +/- 6.5h
280
Target goes below Horizon above/below HA of +/- 6.3h
281
Target goes below Horizon above/below HA of +/- 6.3h
282
Target goes below Horizon a

In [10]:
indexes=np.arange(0,315,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_min.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,21)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"
    VIS_band=metrics.at[i,'VIS band']
    VIS_centre=metrics.at[i,'VIS Centre']
    NIR_centre=metrics.at[i,'NIR Centre']
    guide=metrics.at[i,'Guide']
    if VIS_band == "G":
        VIS_band="V"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[VIS_centre]*u.micron,guide*u.micron)
    VIS_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))

    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[NIR_centre]*u.micron,guide*u.micron)
    NIR_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))
    
    metrics=pd.read_csv("generic_test_cases_final_v2.csv")
    metrics.at[i,'Guide']=guide
    metrics.at[i,'VIS Centre']=VIS_centre
    metrics.at[i,'NIR Centre']=NIR_centre
    metrics.at[i,'VIS Min']=VIS_metrics[0][0]
    metrics.at[i,'VIS Max']=VIS_metrics[0][1]
    metrics.at[i,'VIS Throughput']=VIS_metrics[0][2]
    metrics.at[i,'NIR Min']=NIR_metrics[0][0]
    metrics.at[i,'NIR Max']=NIR_metrics[0][1]
    metrics.at[i,'NIR Throughput']=NIR_metrics[0][2]
    metrics.at[i,'Metric']=NIR_metrics[0][2]**2*NIR_metrics[0][0]*VIS_metrics[0][2]**2*VIS_metrics[0][0]
    metrics.to_csv("generic_test_cases_final_v2.csv",index=False)
  
    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical durham",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[VIS_centre]*u.micron,guide*u.micron)
    VIS_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))
    
    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical durham",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[NIR_centre]*u.micron,guide*u.micron)
    NIR_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))  
    
    metrics=pd.read_csv("generic_test_cases_final_v2.csv")
    metrics.at[315+i,'Guide']=guide
    metrics.at[315+i,'VIS Centre']=VIS_centre
    metrics.at[315+i,'NIR Centre']=NIR_centre
    metrics.at[315+i,'VIS Min']=VIS_metrics[0][0]
    metrics.at[315+i,'VIS Max']=VIS_metrics[0][1]
    metrics.at[315+i,'VIS Throughput']=VIS_metrics[0][2]
    metrics.at[315+i,'NIR Min']=NIR_metrics[0][0]
    metrics.at[315+i,'NIR Max']=NIR_metrics[0][1]
    metrics.at[315+i,'NIR Throughput']=NIR_metrics[0][2]
    metrics.at[315+i,'Metric']=NIR_metrics[0][2]**2*NIR_metrics[0][0]*VIS_metrics[0][2]**2*VIS_metrics[0][0]
    metrics.to_csv("generic_test_cases_min_refined.csv",index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Target goes below Horizon above/below HA of +/- 11.3h
Target goes below Horizon above/below HA of +/- 11.3h
16
Target goes below Horizon above/below HA of +/- 11.3h
Target goes below Horizon above/below HA of +/- 11.3h
17
Target goes below Horizon above/below HA of +/- 11.3h
Target goes below Horizon above/below HA of +/- 11.3h
18
Target goes below Horizon above/below HA of +/- 11.3h
Target goes below Horizon above/below HA of +/- 11.3h
19
Target goes below Horizon above/below HA of +/- 11.3h
Target goes below Horizon above/below HA of +/- 11.3h
20
Target goes below Horizon above/below HA of +/- 9.5h
Target goes below Horizon above/below HA of +/- 9.5h
21
Target goes below Horizon above/below HA of +/- 9.5h
Target goes below Horizon above/below HA of +/- 9.5h
22
Target goes below Horizon above/below HA of +/- 9.5h
Target goes below Horizon above/below HA of +/- 9.5h
23
Target goes below Horizon above/below HA of +/- 9.5h
Target goes below Horizon a

# OTHER METRICS

In [15]:
def optimise_parallel(analysis,guide_options,VIS_aperture_centre_options,NIR_aperture_centre_options,VIS_band,scale_val=0.01):
    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=scale_val)
    VIS_metrics=[] #will be of the form [[[G1 A1 min, max, mean],[G1 A2 min, max, mean]...][[G2 A1 min, max, mean...]...]...]
    #first index is guide, second index is aperture centre, third index in min, max, or mean
    for guide in guide_options:
        integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,VIS_aperture_centre_options,guide)
        VIS_metrics.append(int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission)))
    
    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=scale_val)
    NIR_metrics=[] #will be of the form [[[G1 A1 min, max, mean],[G1 A2 min, max, mean]...][[G2 A1 min, max, mean...]...]...]
    #first index is guide, second index is aperture centre, third index in min, max, or mean
    for guide in guide_options:
        integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,NIR_aperture_centre_options,guide)
        NIR_metrics.append(int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission)))
    
    #Need to go through each guide and each permutation of VIS and NIR
    performance_metrics_v1=[]
    performance_metrics_v2=[]
    performance_metrics_v3=[]
    performance_metrics_v4=[]
    for i in range(0,len(guide_options)):
        metric_outer_v1=[]
        metric_outer_v2=[]
        metric_outer_v3=[]
        metric_outer_v4=[]
        for o in range(0,len(VIS_aperture_centre_options)):
            metric_inner_v1=[]
            metric_inner_v2=[]
            metric_inner_v3=[]
            metric_inner_v4=[]
            for p in range(0,len(NIR_aperture_centre_options)):
                VIS_min=VIS_metrics[i][o][0]
                VIS_max=VIS_metrics[i][o][1]
                VIS_throughput=VIS_metrics[i][o][2]
                
                NIR_min=NIR_metrics[i][p][0]
                NIR_max=NIR_metrics[i][p][1]
                NIR_throughput=NIR_metrics[i][p][2]

                metric_v1=(VIS_min/VIS_max)*VIS_throughput*(NIR_min/NIR_max)*NIR_throughput
                metric_v2=(VIS_min/VIS_max)*VIS_throughput**2*(NIR_min/NIR_max)*NIR_throughput**2
                
                metric_v3=(VIS_min)*VIS_throughput*(NIR_min)*NIR_throughput
                metric_v4=(VIS_min)*VIS_throughput**2*(NIR_min)*NIR_throughput**2 
                             
                metric_inner_v1.append(metric_v1)
                metric_inner_v2.append(metric_v2)
                metric_inner_v3.append(metric_v3)
                metric_inner_v4.append(metric_v4)
                
            metric_outer_v1.append(metric_inner_v1)
            metric_outer_v2.append(metric_inner_v2)
            metric_outer_v3.append(metric_inner_v3)
            metric_outer_v4.append(metric_inner_v4)
                       
        performance_metrics_v1.append(metric_outer_v1)
        performance_metrics_v2.append(metric_outer_v2)
        performance_metrics_v3.append(metric_outer_v3)
        performance_metrics_v4.append(metric_outer_v4)
           
    best_metric_v1=0
    best_metric_v2=0
    best_metric_v3=0
    best_metric_v4=0
    for i in range(0,len(guide_options)):
        for o in range(0,len(VIS_aperture_centre_options)):
            for p in range(0,len(NIR_aperture_centre_options)):
                if performance_metrics_v1[i][o][p]>best_metric_v1:
                    best_metric_v1=performance_metrics_v1[i][o][p]
                    best_guide_v1=guide_options[i]
                    best_VIS_aperture_v1=VIS_aperture_centre_options[o]
                    best_NIR_aperture_v1=NIR_aperture_centre_options[p]
                    
                    best_VIS_metrics_v1=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                    best_NIR_metrics_v1=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
                    
                if performance_metrics_v2[i][o][p]>best_metric_v2:
                    best_metric_v2=performance_metrics_v2[i][o][p]
                    best_guide_v2=guide_options[i]
                    best_VIS_aperture_v2=VIS_aperture_centre_options[o]
                    best_NIR_aperture_v2=NIR_aperture_centre_options[p]
                    
                    best_VIS_metrics_v2=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                    best_NIR_metrics_v2=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
 
                if performance_metrics_v3[i][o][p]>best_metric_v3:
                    best_metric_v3=performance_metrics_v3[i][o][p]
                    best_guide_v3=guide_options[i]
                    best_VIS_aperture_v3=VIS_aperture_centre_options[o]
                    best_NIR_aperture_v3=NIR_aperture_centre_options[p]
                    
                    best_VIS_metrics_v3=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                    best_NIR_metrics_v3=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
                    
                if performance_metrics_v4[i][o][p]>best_metric_v4:
                    best_metric_v4=performance_metrics_v4[i][o][p]
                    best_guide_v4=guide_options[i]
                    best_VIS_aperture_v4=VIS_aperture_centre_options[o]
                    best_NIR_aperture_v4=NIR_aperture_centre_options[p]
                    
                    best_VIS_metrics_v4=[VIS_metrics[i][o][0],VIS_metrics[i][o][1],VIS_metrics[i][o][2]]
                    best_NIR_metrics_v4=[NIR_metrics[i][p][0],NIR_metrics[i][p][1],NIR_metrics[i][p][2]]
                          
    best_metric_combo_v1=[best_metric_v1,best_VIS_metrics_v1,best_NIR_metrics_v1]
    best_metric_combo_v2=[best_metric_v2,best_VIS_metrics_v2,best_NIR_metrics_v2]
    best_metric_combo_v3=[best_metric_v3,best_VIS_metrics_v3,best_NIR_metrics_v3]
    best_metric_combo_v4=[best_metric_v4,best_VIS_metrics_v4,best_NIR_metrics_v4]
    
    return [best_guide_v1,best_VIS_aperture_v1,best_NIR_aperture_v1,best_metric_combo_v1],[best_guide_v2,best_VIS_aperture_v2,best_NIR_aperture_v2,best_metric_combo_v2],[best_guide_v3,best_VIS_aperture_v3,best_NIR_aperture_v3,best_metric_combo_v3],[best_guide_v4,best_VIS_aperture_v4,best_NIR_aperture_v4,best_metric_combo_v4]

In [ ]:
import pandas as pd
#V boundaries: .6 to 1, .9 to 1.4
#R boundaries: .7 to 1, .9 to 1.4
VIS_B=[.390,.458]
VIS_V=[.450,.591] 
VIS_R=[.586,.770]
NIR_All=[.770,1.857]
#indexes=metrics.loc[(metrics['HA mid']==0) & (metrics['VIS band'] == 'B')].index[0:2]
indexes=np.arange(210,315,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_v1.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,11)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    v1,v2,v3,v4=optimise_parallel(analysis,np.arange(.7,1,.01)*u.micron,np.arange(VIS_R[0],VIS_R[1],.01)*u.micron,np.arange(.9,1.4,.01)*u.micron,'R',scale_val=0.1)

    best_guide_v1,best_VIS_aperture_v1,best_NIR_aperture_v1,best_metric_combo_v1=v1[0],v1[1],v1[2],v1[3]
    best_guide_v2,best_VIS_aperture_v2,best_NIR_aperture_v2,best_metric_combo_v2=v2[0],v2[1],v2[2],v2[3]
    best_guide_v3,best_VIS_aperture_v3,best_NIR_aperture_v3,best_metric_combo_v3=v3[0],v3[1],v3[2],v3[3]
    best_guide_v4,best_VIS_aperture_v4,best_NIR_aperture_v4,best_metric_combo_v4=v4[0],v4[1],v4[2],v4[3]
    
    aperture_VIS_v1=[round(best_VIS_aperture_v1.value,2)]*u.micron
    aperture_NIR_v1=[round(best_NIR_aperture_v1.value,2)]*u.micron
    guide_v1=round(best_guide_v1.value,2)*u.micron

    aperture_VIS_v2=[round(best_VIS_aperture_v2.value,2)]*u.micron
    aperture_NIR_v2=[round(best_NIR_aperture_v2.value,2)]*u.micron
    guide_v2=round(best_guide_v2.value,2)*u.micron
    
    aperture_VIS_v3=[round(best_VIS_aperture_v3.value,2)]*u.micron
    aperture_NIR_v3=[round(best_NIR_aperture_v3.value,2)]*u.micron
    guide_v3=round(best_guide_v3.value,2)*u.micron

    aperture_VIS_v4=[round(best_VIS_aperture_v4.value,2)]*u.micron
    aperture_NIR_v4=[round(best_NIR_aperture_v4.value,2)]*u.micron
    guide_v4=round(best_guide_v4.value,2)*u.micron   
       
    metrics.at[i,'Guide']=guide_v1.value
    metrics.at[i,'VIS Centre']=aperture_VIS_v1.value
    metrics.at[i,'NIR Centre']=aperture_NIR_v1.value
    metrics.at[i,'VIS Min']=best_metric_combo_v1[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo_v1[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo_v1[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo_v1[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo_v1[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo_v1[2][2]
    metrics.at[i,'Metric']=best_metric_combo_v1[0]
    metrics.to_csv("generic_test_cases_v1.csv",index=False)
    
    metrics=pd.read_csv("generic_test_cases_v2.csv")
    metrics.at[i,'Guide']=guide_v2.value
    metrics.at[i,'VIS Centre']=aperture_VIS_v2.value
    metrics.at[i,'NIR Centre']=aperture_NIR_v2.value
    metrics.at[i,'VIS Min']=best_metric_combo_v2[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo_v2[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo_v2[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo_v2[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo_v2[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo_v2[2][2]
    metrics.at[i,'Metric']=best_metric_combo_v2[0]
    metrics.to_csv("generic_test_cases_v2.csv",index=False)

    metrics=pd.read_csv("generic_test_cases_v3.csv")
    metrics.at[i,'Guide']=guide_v3.value
    metrics.at[i,'VIS Centre']=aperture_VIS_v3.value
    metrics.at[i,'NIR Centre']=aperture_NIR_v3.value
    metrics.at[i,'VIS Min']=best_metric_combo_v3[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo_v3[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo_v3[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo_v3[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo_v3[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo_v3[2][2]
    metrics.at[i,'Metric']=best_metric_combo_v3[0]
    metrics.to_csv("generic_test_cases_v3.csv",index=False)

    metrics=pd.read_csv("generic_test_cases_v4.csv")
    metrics.at[i,'Guide']=guide_v4.value
    metrics.at[i,'VIS Centre']=aperture_VIS_v4.value
    metrics.at[i,'NIR Centre']=aperture_NIR_v4.value
    metrics.at[i,'VIS Min']=best_metric_combo_v4[1][0]
    metrics.at[i,'VIS Max']=best_metric_combo_v4[1][1]
    metrics.at[i,'VIS Throughput']=best_metric_combo_v4[1][2]
    metrics.at[i,'NIR Min']=best_metric_combo_v4[2][0]
    metrics.at[i,'NIR Max']=best_metric_combo_v4[2][1]
    metrics.at[i,'NIR Throughput']=best_metric_combo_v4[2][2]
    metrics.at[i,'Metric']=best_metric_combo_v4[0]
    metrics.to_csv("generic_test_cases_v4.csv",index=False)

In [ ]:
indexes=np.arange(0,315,1)
for i in indexes:
    metrics=pd.read_csv("generic_test_cases_v4.csv")
    print(i)
    HA_range=np.linspace(metrics.at[i,'HA mid']-0.5,metrics.at[i,'HA mid']+0.5,21)
    targ_dec=metrics.at[i,'Dec']*u.deg
    centring="mid HA"
    VIS_band=metrics.at[i,'VIS band']
    VIS_centre=metrics.at[i,'VIS Centre']
    NIR_centre=metrics.at[i,'NIR Centre']
    guide=metrics.at[i,'Guide']
    if VIS_band == "G":
        VIS_band="V"

    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[VIS_centre]*u.micron,guide*u.micron)
    VIS_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))

    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical moffat",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[NIR_centre]*u.micron,guide*u.micron)
    NIR_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))
    
    metrics=pd.read_csv("generic_test_cases_final.csv")
    metrics.at[i,'Guide']=guide
    metrics.at[i,'VIS Centre']=VIS_centre
    metrics.at[i,'NIR Centre']=NIR_centre
    metrics.at[i,'VIS Min']=VIS_metrics[0][0]
    metrics.at[i,'VIS Max']=VIS_metrics[0][1]
    metrics.at[i,'VIS Throughput']=VIS_metrics[0][2]
    metrics.at[i,'NIR Min']=NIR_metrics[0][0]
    metrics.at[i,'NIR Max']=NIR_metrics[0][1]
    metrics.at[i,'NIR Throughput']=NIR_metrics[0][2]
    metrics.at[i,'Metric']=NIR_metrics[0][2]**2*NIR_metrics[0][0]*VIS_metrics[0][2]**2*VIS_metrics[0][0]
    metrics.to_csv("generic_test_cases_final.csv",index=False)
  
    analysis=AD_analysis()
    analysis.load_airmasses(HA_range=HA_range,targ_dec=targ_dec)

    analysis.load_band('LR','VIS',VIS_band,VIS_band,sampling=1*u.nm)
    analysis.load_wave(.702*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical durham",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[VIS_centre]*u.micron,guide*u.micron)
    VIS_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))
    
    analysis.load_band('LR','NIR','IY','H',sampling=1*u.nm) 
    analysis.load_wave(.570*u.arcsec,min(analysis.output['wavelengths']).value,max(analysis.output['wavelengths']).value,segments=10)
    analysis.make_aperture("hexagons",method="numerical durham",scale=0.01)

    integ_transmissions,opt_transmission=int_analysis.integ_trans(analysis,[NIR_centre]*u.micron,guide*u.micron)
    NIR_metrics=int_analysis.integ_metric(np.array(integ_transmissions)/np.array(opt_transmission))  
    
    metrics=pd.read_csv("generic_test_cases_final.csv")
    metrics.at[315+i,'Guide']=guide
    metrics.at[315+i,'VIS Centre']=VIS_centre
    metrics.at[315+i,'NIR Centre']=NIR_centre
    metrics.at[315+i,'VIS Min']=VIS_metrics[0][0]
    metrics.at[315+i,'VIS Max']=VIS_metrics[0][1]
    metrics.at[315+i,'VIS Throughput']=VIS_metrics[0][2]
    metrics.at[315+i,'NIR Min']=NIR_metrics[0][0]
    metrics.at[315+i,'NIR Max']=NIR_metrics[0][1]
    metrics.at[315+i,'NIR Throughput']=NIR_metrics[0][2]
    metrics.at[315+i,'Metric']=NIR_metrics[0][2]**2*NIR_metrics[0][0]*VIS_metrics[0][2]**2*VIS_metrics[0][0]
    metrics.to_csv("generic_test_cases_final.csv",index=False)